## Dump the join logs first then run the cells, changing 'credits' and 'ratings' accordingly.

## How to dump all the logs:

Run in the terminal:
```bash
for i in $(seq 1 5); do docker logs join_credits_$i >& cred$i.log; docker logs join_ratings_$i >& rat$i.log; done
```

Then run the cells below to read the logs and create the DataFrames.

In [39]:
import re
import pandas as pd
from pathlib import Path

join = 'credits'
#join = 'ratings'

if join == 'credits':
    file_prefix = 'cred'
    log = '[Join-Credits]'
else:
    file_prefix = 'rat'
    log = '[Join-Ratings]'


# Define the regex to extract the required fields
pattern = re.compile(
    r'Message ID: (\d+) .*? for client ([a-f0-9-]+) has joined data of length (\d+)\.', re.IGNORECASE
)

# Function to parse one file and extract relevant data
def extract_joined_data(filepath):
    data = []
    node_match = re.search(rf'{file_prefix}(\d+)\.log$', filepath.name)
    if not node_match:
        print(f"Skipping file {filepath} as it does not match expected pattern.")
        return data
    node = int(node_match.group(1)) if node_match else None

    with open(filepath, 'r', encoding='utf-8', errors='ignore') as f:
        print(f"Processing file: {filepath}")
        for line in f:
            if log in line:
                match = pattern.search(line)
                if match:
                    message_id = int(match.group(1))
                    client_id = match.group(2)
                    joined_length = int(match.group(3))

                    # Also extract the total original length from earlier in the line
                    length_match = re.search(r'length (\d+)', line)
                    original_length = int(length_match.group(1)) if length_match else None

                    data.append({
                        'node': node,
                        'message_id': message_id,
                        'client_id': client_id,
                        'original_length': original_length,
                        'joined_length': joined_length,
                    })
    return data

# Load all relevant log files
log_files = list(Path('.').glob(f'{file_prefix}*.log'))

print(f"Found {len(log_files)} log files matching pattern '{file_prefix}*.log'.")

# Extract and combine all the data
all_data = []
for log_file in log_files:
    all_data.extend(extract_joined_data(log_file))

# Convert to DataFrame
df = pd.DataFrame(all_data)


Found 5 log files matching pattern 'cred*.log'.
Processing file: cred1.log
Processing file: cred2.log
Processing file: cred3.log
Processing file: cred4.log
Processing file: cred5.log


In [40]:
df

,node,message_id,client_id,original_length,joined_length
0,1,64,4a16a597-9cda-410b-b55e-bff40e2d10e4,180,1
1,1,74,4a16a597-9cda-410b-b55e-bff40e2d10e4,148,1
2,1,77,4a16a597-9cda-410b-b55e-bff40e2d10e4,144,1
3,1,78,4a16a597-9cda-410b-b55e-bff40e2d10e4,150,1
4,1,84,4a16a597-9cda-410b-b55e-bff40e2d10e4,208,1
...,...,...,...,...,...
297,5,212,4a16a597-9cda-410b-b55e-bff40e2d10e4,67,3
298,5,194,d776118c-cfdf-4dd1-be6e-25968a96ae46,28,3
299,5,200,d776118c-cfdf-4dd1-be6e-25968a96ae46,69,1
300,5,206,d776118c-cfdf-4dd1-be6e-25968a96ae46,126,1


In [42]:
import pandas as pd

# Assumes `df` exists with: client_id, message_id, original_length, joined_length, node

# Step 1: Create a unique key for comparison
df['msg_key'] = list(zip(df['message_id'], df['original_length'], df['joined_length']))

# Step 2: Create sets of msg_keys per (client, node)
grouped = df.groupby(['client_id', 'node'])['msg_key'].apply(set).reset_index()

# Step 3: Pivot to have clients as columns, nodes as rows
pivoted = grouped.pivot(index='node', columns='client_id', values='msg_key')

# Step 4: Compare all client sets pairwise per node
def compare_clients(row):
    sets = [s for s in row if isinstance(s, set)]
    return all(s == sets[0] for s in sets[1:])

pivoted['all_clients_equal'] = pivoted.apply(compare_clients, axis=1)

# Display nodes where clients differ
inconsistent_nodes = pivoted[~pivoted['all_clients_equal']]
print(f"⚠️ {len(inconsistent_nodes)} nodes where clients saw different message data:")
inconsistent_nodes


⚠️ 0 nodes where clients saw different message data:


client_id,4a16a597-9cda-410b-b55e-bff40e2d10e4,d776118c-cfdf-4dd1-be6e-25968a96ae46,all_clients_equal
node,,,


In [ ]:
# Choose a specific node to inspect (e.g., node 2)
node_to_check = 1
client_sets = pivoted.loc[node_to_check, pivoted.columns != 'all_clients_equal']

# Find pairwise differences
clients = client_sets.index.tolist()
for i in range(len(clients)):
    for j in range(i+1, len(clients)):
        c1, c2 = clients[i], clients[j]
        only_c1 = client_sets[c1] - client_sets[c2]
        only_c2 = client_sets[c2] - client_sets[c1]
        print(f"\nBetween {c1} and {c2} on node {node_to_check}:")
        print(f"  ✅ Shared: {len(client_sets[c1] & client_sets[c2])}")
        print(f"  ❌ Only in {c1}: {only_c1}")
        print(f"  ❌ Only in {c2}: {only_c2}")


Between 3df84a97-7358-4202-9721-a445e8fef206 and fdeb2329-64cd-4805-9f62-93b098f50f0c on node 3:
  ✅ Shared: 18
  ❌ Only in 3df84a97-7358-4202-9721-a445e8fef206: set()
  ❌ Only in fdeb2329-64cd-4805-9f62-93b098f50f0c: set()


In [ ]:
# Choose a specific node to inspect (e.g., node 2)
node_to_check = 2
client_sets = pivoted.loc[node_to_check, pivoted.columns != 'all_clients_equal']

# Find pairwise differences
clients = client_sets.index.tolist()
for i in range(len(clients)):
    for j in range(i+1, len(clients)):
        c1, c2 = clients[i], clients[j]
        only_c1 = client_sets[c1] - client_sets[c2]
        only_c2 = client_sets[c2] - client_sets[c1]
        print(f"\nBetween {c1} and {c2} on node {node_to_check}:")
        print(f"  ✅ Shared: {len(client_sets[c1] & client_sets[c2])}")
        print(f"  ❌ Only in {c1}: {only_c1}")
        print(f"  ❌ Only in {c2}: {only_c2}")